# DataFrames.jl

In [1]:
using DataFrames, Statistics

In [2]:
df = DataFrame(id = 'a':'f', grp = repeat(1:2, 3), x = 6:-1:1, y = 4:9, z = [3:7; missing])

,id,grp,x,y,z
,Char,Int64,Int64,Int64,Int64?
1,'a',1,6,4,3
2,'b',2,5,5,4
3,'c',1,4,6,5
4,'d',2,3,7,6
5,'e',1,2,8,7
6,'f',2,1,9,missing


DataFrames.jl does not support row indices. In this example, the record identifiers are stored as in the `id` column. 

# Accessing data

Julia uses 1-based indexing.

In [3]:
df[2, 2]  # second row, second column.

2

In [4]:
df[2:3, :]  # second and third rows, all columns

,id,grp,x,y,z
,Char,Int64,Int64,Int64,Int64?
1,'b',2,5,5,4
2,'c',1,4,6,5


In [5]:
df[:, 2:end]   # all rows, second to last columns

,grp,x,y,z
,Int64,Int64,Int64,Int64?
1,1,6,4,3
2,2,5,5,4
3,1,4,6,5
4,2,3,7,6
5,1,2,8,7
6,2,1,9,missing


DataFrames.jl indexing using `findfirst` always return a DataFrameRow object. To match multiple rows, use `findall` and it would return a DataFrame object.

In [6]:
df[findfirst(==('c'), df.id), :]   # single row having id == 'c'

,id,grp,x,y,z
,Char,Int64,Int64,Int64,Int64?
3,'c',1,4,6,5


In [7]:
df[:, [:id, :x]]    # all rows, 'x' column    

,id,x
,Char,Int64
1,'a',6
2,'b',5
3,'c',4
4,'d',3
5,'e',2
6,'f',1


In [8]:
df[:, [:id, :x, :z]]   # all rows, columns 'x' and 'z'

,id,x,z
,Char,Int64,Int64?
1,'a',6,3
2,'b',5,4
3,'c',4,5
4,'d',3,6
5,'e',2,7
6,'f',1,missing


In [9]:
# df[:, [:id, Between(:x, :z)]]    # direct indexing does not work in this case
select(df, :id, Between(:x, :z))   # all rows, alls columns between 'x' and 'z'

,id,x,y,z
,Char,Int64,Int64,Int64?
1,'a',6,4,3
2,'b',5,5,4
3,'c',4,6,5
4,'d',3,7,6
5,'e',2,8,7
6,'f',1,9,missing


In [10]:
df[findfirst(==('c'), df.id), 3]   # row having id = 'c', third column

4

# Common operations

In [11]:
mean(df.z)

missing

In [12]:
mean(skipmissing(df.z))

5.0

In [13]:
combine(df, :z => mean ∘ skipmissing)

,z_mean_skipmissing
,Float64
1,5.0


In [14]:
transform(df, :z => (v -> v .+ 1) => :z1)

,id,grp,x,y,z,z1
,Char,Int64,Int64,Int64,Int64?,Int64?
1,'a',1,6,4,3,4
2,'b',2,5,5,4,5
3,'c',1,4,6,5,6
4,'d',2,3,7,6,7
5,'e',1,2,8,7,8
6,'f',2,1,9,missing,missing


In [15]:
rename(df, :x => :x_new)

,id,grp,x_new,y,z
,Char,Int64,Int64,Int64,Int64?
1,'a',1,6,4,3
2,'b',2,5,5,4
3,'c',1,4,6,5
4,'d',2,3,7,6
5,'e',1,2,8,7
6,'f',2,1,9,missing


In [16]:
select(df, :id, :x => mean, :y)

,id,x_mean,y
,Char,Float64,Int64
1,'a',3.5,4
2,'b',3.5,5
3,'c',3.5,6
4,'d',3.5,7
5,'e',3.5,8
6,'f',3.5,9


In [17]:
sort(df, :x)

,id,grp,x,y,z
,Char,Int64,Int64,Int64,Int64?
1,'f',2,1,9,missing
2,'e',1,2,8,7
3,'d',2,3,7,6
4,'c',1,4,6,5
5,'b',2,5,5,4
6,'a',1,6,4,3


In [18]:
sort(df, [:grp, order(:x, rev = true)])

,id,grp,x,y,z
,Char,Int64,Int64,Int64,Int64?
1,'a',1,6,4,3
2,'c',1,4,6,5
3,'e',1,2,8,7
4,'b',2,5,5,4
5,'d',2,3,7,6
6,'f',2,1,9,missing


# Grouping data and aggregation

In [19]:
combine(groupby(df, :grp), :x => mean)

,grp,x_mean
,Int64,Float64
1,1,4.0
2,2,3.0


In [20]:
combine(groupby(df, :grp), :x => mean => :my_mean)

,grp,my_mean
,Int64,Float64
1,1,4.0
2,2,3.0


In [21]:
transform(groupby(df, :grp), :x => mean)

,id,grp,x,y,z,x_mean
,Char,Int64,Int64,Int64,Int64?,Float64
1,'a',1,6,4,3,4.0
2,'b',2,5,5,4,3.0
3,'c',1,4,6,5,4.0
4,'d',2,3,7,6,3.0
5,'e',1,2,8,7,4.0
6,'f',2,1,9,missing,3.0


In [22]:
select(groupby(df, :grp), :id, :x => mean)

,grp,id,x_mean
,Int64,Char,Float64
1,1,'a',4.0
2,2,'b',3.0
3,1,'c',4.0
4,2,'d',3.0
5,1,'e',4.0
6,2,'f',3.0


# More advanced commands

In [23]:
combine(df, :z => v -> mean(cos, skipmissing(v)))

,z_function
,Float64
1,0.0708197


In [24]:
combine(df, :x => maximum, :y => minimum)

,x_maximum,y_minimum
,Int64,Int64
1,6,4


In [25]:
combine(df, [:x, :y] .=> mean)

,x_mean,y_mean
,Float64,Float64
1,3.5,6.5


In [26]:
combine(v -> mapcols(mean, v), select(df, r"[xy]"))

,x,y
,Float64,Float64
1,3.5,6.5


In [27]:
# combine(df, ([:x, :y] .=> [maximum minimum])...)  # this doesn't match pandas' shape
DataFrame([(agg="$f", x=f(df.x), y=f(df.y)) for f in [maximum, minimum]])

,agg,x,y
,String,Int64,Int64
1,maximum,6,9
2,minimum,1,4


In [28]:
transform(df, [:x, :y] => cor)

,id,grp,x,y,z,x_y_cor
,Char,Int64,Int64,Int64,Int64?,Float64
1,'a',1,6,4,3,-1.0
2,'b',2,5,5,4,-1.0
3,'c',1,4,6,5,-1.0
4,'d',2,3,7,6,-1.0
5,'e',1,2,8,7,-1.0
6,'f',2,1,9,missing,-1.0


In [29]:
transform(df, [:x, :y] => ByRow(min))

,id,grp,x,y,z,x_y_min
,Char,Int64,Int64,Int64,Int64?,Int64
1,'a',1,6,4,3,4
2,'b',2,5,5,4,5
3,'c',1,4,6,5,4
4,'d',2,3,7,6,3
5,'e',1,2,8,7,2
6,'f',2,1,9,missing,1


In [30]:
transform(df, AsTable([:x,:y]) => ByRow(argmax))

,id,grp,x,y,z,x_y_argmax
,Char,Int64,Int64,Int64,Int64?,Symbol
1,'a',1,6,4,3,x
2,'b',2,5,5,4,x
3,'c',1,4,6,5,y
4,'d',2,3,7,6,y
5,'e',1,2,8,7,y
6,'f',2,1,9,missing,y


In [31]:
combine(d -> first(d, 2), groupby(df, :grp))

,grp,id,x,y,z
,Int64,Char,Int64,Int64,Int64?
1,1,'a',6,4,3
2,1,'c',4,6,5
3,2,'b',5,5,4
4,2,'d',3,7,6


In [32]:
combine(:x => x -> (x = [minimum(x), maximum(x)],), df)

,x
,Int64
1,1
2,6


# Joining data frames

In [33]:
# Preparation
df2 = DataFrame(grp = [1, 3], w = [10, 11])

,grp,w
,Int64,Int64
1,1,10
2,3,11


In [34]:
innerjoin(df, df2, on = :grp)

,id,grp,x,y,z,w
,Char,Int64,Int64,Int64,Int64?,Int64
1,'a',1,6,4,3,10
2,'c',1,4,6,5,10
3,'e',1,2,8,7,10


In [35]:
outerjoin(df, df2, on = :grp)

,id,grp,x,y,z,w
,Char?,Int64,Int64?,Int64?,Int64?,Int64?
1,'a',1,6,4,3,10
2,'b',2,5,5,4,missing
3,'c',1,4,6,5,10
4,'d',2,3,7,6,missing
5,'e',1,2,8,7,10
6,'f',2,1,9,missing,missing
7,missing,3,missing,missing,missing,11


In [36]:
leftjoin(df, df2, on = :grp)

,id,grp,x,y,z,w
,Char,Int64,Int64,Int64,Int64?,Int64?
1,'a',1,6,4,3,10
2,'b',2,5,5,4,missing
3,'c',1,4,6,5,10
4,'d',2,3,7,6,missing
5,'e',1,2,8,7,10
6,'f',2,1,9,missing,missing


In [37]:
rightjoin(df, df2, on = :grp)

,id,grp,x,y,z,w
,Char?,Int64,Int64?,Int64?,Int64?,Int64
1,'a',1,6,4,3,10
2,'c',1,4,6,5,10
3,'e',1,2,8,7,10
4,missing,3,missing,missing,missing,11


In [38]:
semijoin(df, df2, on = :grp)

,id,grp,x,y,z
,Char,Int64,Int64,Int64,Int64?
1,'a',1,6,4,3
2,'c',1,4,6,5
3,'e',1,2,8,7


In [39]:
antijoin(df, df2, on = :grp)

,id,grp,x,y,z
,Char,Int64,Int64,Int64,Int64?
1,'b',2,5,5,4
2,'d',2,3,7,6
3,'f',2,1,9,missing
